OVERVIEW

This project analyzes a house sales dataset in King County, USA, to predicts housing prices using Apache Spark’s MLlib library to build different regression models through the PySpark API. The optimal model was chosen based on the least Root Mean Squared Error and highest R2

ABOUT DATASET

The dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015 and it's a good dataset for evaluating simple regression models.

| Variable      | Description                                                                                                 |
| ------------- | ----------------------------------------------------------------------------------------------------------- |
| id            | A notation for a house                                                                                      |
| date          | Date house was sold                                                                                         |
| price         | Price is prediction target                                                                                  |
| bedrooms      | Number of bedrooms                                                                                          |
| bathrooms     | Number of bathrooms                                                                                         |
| sqft_living   | Square footage of the home                                                                                  |
| sqft_lot      | Square footage of the lot                                                                                   |
| floors        | Total floors (levels) in house                                                                              |
| waterfront    | House which has a view to a waterfront                                                                      |
| view          | Has been viewed                                                                                             |
| condition     | How good the condition is overall                                                                           |
| grade         | overall grade given to the housing unit, based on King County grading system                                |
| sqft_above    | Square footage of house apart from basement                                                                 |
| sqft_basement | Square footage of the basement                                                                              |
| yr_built      | Built Year                                                                                                  |
| yr_renovated  | Year when house was renovated                                                                               |
| zipcode       | Zip code                                                                                                    |
| lat           | Latitude coordinate                                                                                         |
| long          | Longitude coordinate                                                                                        |
| sqft_living15 | Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area |
| sqft_lot15    | LotSize area in 2015(implies-- some renovations)                                                            |


SETUP

We set up Spark environment to initialize SparkSession and import the required libraries. 

In [ ]:
## importing required libraries
# to find spark
import findspark # to run PySpark in Jupyter notebook
findspark.init()
findspark.find()

# import spark
from pyspark.sql import SparkSession      # To create a Spark session, the entry point for using Spark functionality
from pyspark.sql.functions import regexp_replace, col, substring
from pyspark.sql.functions import corr    # Calculates the Pearson correlation coefficient between two columns
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor # Provides Regression models
from pyspark.ml.feature import VectorAssembler     # Combines multiple columns into a single vector column, required for MLlib models
from pyspark.ml.evaluation import RegressionEvaluator # Evaluates the performance of regression models


# initialize SparkSession
spark = SparkSession.builder \
    .appName('Regression with PySpark MLlib') \
        .getOrCreate()

LOAD DATASET

Set the header and inferSchema parameters to be True, which means the first line contains the header and automatic detection of the underlying data schema respectively

In [183]:
# load the dataset
df = spark.read.csv("kc_house_data.csv", inferSchema=True, header=True)

In [184]:
# check the dataset
df.show(5)

+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|   

We can notice that the target variable (price) in the third column. As a good practice, let us rearrange columns and make target variable to be last

In [185]:
# rearrange dataset column
rearranged_col = ["id", "date", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view", "condition", "grade", "sqft_above", "sqft_basement", "yr_built",  "yr_renovated", "zipcode", "lat", "long", "sqft_living15", "sqft_lot15", "price"]
df = df.select(rearranged_col)
df.show(5)

+----------+---------------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------+
|        id|           date|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|   price|
+----------+---------------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------+
|7129300520|20141013T000000|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|221900.0|
|6414100192|20141209T000000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|   

DATA CLEANING AND PREPROCESSING

Checking the Types of Data

Let us check for the datatype because sometimes variables can be stored in wrong format.

In [186]:
# check data types
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)
 |-- price: double (nullable = true)



In this case, it appears all features as stored correctly so nothing to worry about.

Let us move on to dropping irrelevant columns. This step is certainly required because sometimes there would be columns that will not be needed, in such cases dropping is the right thing to do. In this case, the “id” column will not contribute any value to our analysis so we will be dropping it.


In [187]:
# Drop irrelevant column 
df = df.drop("id")
df.show(5)

+---------------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------+
|           date|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|   price|
+---------------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+--------+
|20141013T000000|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|221900.0|
|20141209T000000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125| 47.721|-122.319|         1690|      

Formatting Columns for Numerical Analysis

Let's extract the first 4 characters of the 'date' column to format it as year this will ensure all our features are in numerical type which is good for our analysis.


In [188]:
# Let's extract the first 4 characters of the 'date' column to format it as year
# this will ensure all our features are in numerical type which is good for our analysis
df = df.withColumn("date", substring(col("date"), 1, 4).cast("int"))

# Rename the 'date' column to 'yr_sold'
df = df.withColumnRenamed("date", "yr_sold")

Now that all our features are in numeric let us obtain Statistical summary from the data

In [189]:
# Statistical summary
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-----------------+------------------+------------------+
|summary|           yr_sold|         bedrooms|         bathrooms|       sqft_living|          sqft_lot|            floors|          waterfront|               view|         condition|             grade|        sqft_above|     sqft_basement|          yr_built|      yr_renovated|           zipcode|                lat|               long|    sqft_living15|        sqft_lot15|             price|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+-------------

Checking and Dropping Duplicate Rows

This is often a useful thing to do because a huge data set like in this case sometimes have some duplicate data which can impact analysis result, so we remove all the duplicate value from the dataset.


In [190]:
# check and drop duplicate rows
print("Duplicate rows before:", df.count() - df.distinct().count()) # Check duplicate rows before dropping

# Drop duplicate rows
df = df.dropDuplicates()

print("Duplicate rows after:", df.count() - df.distinct().count()) # Check duplicate rows after dropping

# statistical summary after dropping duplicates 
df.describe().show()

Duplicate rows before: 4
Duplicate rows after: 0
+-------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------------+-------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+-------------------+-------------------+------------------+------------------+-----------------+
|summary|           yr_sold|          bedrooms|         bathrooms|      sqft_living|         sqft_lot|            floors|          waterfront|               view|         condition|            grade|        sqft_above|     sqft_basement|         yr_built|     yr_renovated|          zipcode|                lat|               long|     sqft_living15|        sqft_lot15|            price|
+-------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------------+-------------------+-------------

Before removing the duplicates, there were 21,613 rows of data but after, there are data 21,609 meaning there were 4 of duplicate data as shown in the code snippet below.

Transforming Features

The MLlib algorithms expect the data to be represented in two columns: Features and Label. Features are an array of data points of all the features to be used for prediction while the Label contain the output label for each data point.

Before building the model, we will assemble the input features into a single feature vector using the VectorAssembler class, transform the data and then select the features and the label (price).


In [191]:
# create a VectorAssembler for a single feature
assembler = VectorAssembler(
    inputCols=["zipcode", "yr_sold", "long", "condition", "yr_built", "sqft_lot15", "sqft_lot", "yr_renovated", "floors", 
               "waterfront", "lat", "bedrooms", "sqft_basement", "view", "bathrooms", "sqft_living15", "sqft_above", "grade",
                 "sqft_living"], 
                 outputCol="feature")

# Transform the data
df = assembler.transform(df)

# Select feature and label (predictor and target features)
df_final = df.select("feature", "price")

Now we will split the data into a training set (70%) and a testing set (30%) using randomSplit() function to select random rows 

In [192]:
# Now we will split the data into a training set (70%) and a testing set (30%)
df_train, df_test = df_final.randomSplit([0.7, 0.3])
print('Train data:')
df_train.show(5)

print('Test data:')
df_test.show(5)

Train data:
+--------------------+--------+
|             feature|   price|
+--------------------+--------+
|[98001.0,2014.0,-...|290000.0|
|[98001.0,2014.0,-...|420000.0|
|[98001.0,2014.0,-...|214100.0|
|[98001.0,2014.0,-...|335000.0|
|[98001.0,2014.0,-...|199900.0|
+--------------------+--------+
only showing top 5 rows

Test data:
+--------------------+--------+
|             feature|   price|
+--------------------+--------+
|[98001.0,2014.0,-...|850000.0|
|[98001.0,2014.0,-...|571500.0|
|[98001.0,2014.0,-...|310000.0|
|[98001.0,2014.0,-...|280000.0|
|[98001.0,2014.0,-...|185000.0|
+--------------------+--------+
only showing top 5 rows



MODELS DEVELOPMENT

Next, let us create instances of the regression models we are interested in and fit the models to the training data.

In [193]:
# create an object of the LinearRegression model
lr = LinearRegression(featuresCol='feature', labelCol='price')

# create an object of the DecisionTreeRegressor model
dt = DecisionTreeRegressor(featuresCol='feature', labelCol='price')

# create an object of the RandomForestRegressor algorithm
rf = RandomForestRegressor(featuresCol='feature', labelCol='price', numTrees=2, maxDepth=2, seed=42)

# fit the models to the training data.
model = lr.fit(df_train)
model_2 = dt.fit(df_train)
model_3 = rf.fit(df_train)

PREDICTION AND EVALUATION

We will use the trained models to make predictions on the test data and evaluate each model's performance to identify the best model. 

There are different model evaluation metrics, to calculate the accuracy of each of the model, will be using the R-squared (R2) and Root Mean Squared Error (RMSE) metrics.

In [194]:
# make prediction
predictions = model.transform(df_test)
predictions_2 = model_2.transform(df_test)
predictions_3 = model_3.transform(df_test)

In [195]:
# Linear Regression
print("Linear Regression Model:")
# R2 score
evaluator_r2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R2 Score: {:.2f}".format(r2))
# root mean squared error 
evaluator_rmse = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse))

# DecisionTreeRegressor
print('')
print("DecisionTreeRegressor Model:")
# R2 score
evaluator_r2_2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2_2 = evaluator_r2_2.evaluate(predictions_2)
print("R2 Score: {:.2f}".format(r2_2))
# root mean squared error 
evaluator_rmse_2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse_2 = evaluator_rmse.evaluate(predictions_2)
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse_2))

# RandomForestRegressor
print('')
print("RandomForestRegressor:")
# R2 score
evaluator_r2_3 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2_3 = evaluator_r2_3.evaluate(predictions_3)
print("R2 Score: {:.2f}".format(r2_3))
# root mean squared error
evaluator_rmse_3 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse_3 = evaluator_rmse.evaluate(predictions_3)
print("Root Mean Squared Error (RMSE): {:.2f}".format(rmse_3))

Linear Regression Model:
R2 Score: 0.71
Root Mean Squared Error (RMSE): 186479.27

DecisionTreeRegressor Model:
R2 Score: 0.72
Root Mean Squared Error (RMSE): 182072.83

RandomForestRegressor:
R2 Score: 0.53
Root Mean Squared Error (RMSE): 237136.10


Moedel 1: Linear Regression Model

With a R2 Score of 0.71 and RMSE of 186,479.27, this model explains 71% of the variance in the data and the RMSE seems moderate, indicating some prediction error.

Model 2: DecisionTreeRegressor Model

With a R2 Score of 0.72 and RMSE of 182,072.83, this model slightly outperforms the Linear Regression model in explaining the variance. Additionally, it has the lowest RMSE among the models, suggesting it provides the most accurate predictions overall.

Model 3: RandomForestRegressor Model

With a R2 Score of 0.53 and RMSE of 237,136.10, this model explains only 53% of the variance and has the highest RMSE, indicating the least accurate predictions among the three.

CONCLUSION

Comparing these three models, we conclude that the DecisionTreeRegressor Model is the best model to be able to predict price from our dataset. it has the highest R2 score (0.72) and the lowest RMSE (182,072.83), balanceing variance explanation with predictive accuracy.